In [1]:
pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 66.7 MB/s eta 0:00:00
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

23/07/12 21:42:20 WARN Utils: Your hostname, lucas-Estudos resolves to a loopback address: 127.0.1.1; using 192.168.0.177 instead (on interface wlp8s0)
23/07/12 21:42:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/12 21:42:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.json("../../datalake/twitter_datascience")

In [6]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{24, 37, 2023-07...|{[{2023-07-10T04:...|{1234567890abcdef}|  2023-07-10|
|[{60, 37, 2023-07...|{[{2023-07-10T12:...|{1234567890abcdef}|  2023-07-10|
|[{97, 91, 2023-07...|{[{2023-07-10T11:...|              null|  2023-07-10|
|[{60, 17, 2023-07...|{[{2023-07-08T14:...|{1234567890abcdef}|  2023-07-08|
|[{89, 37, 2023-07...|{[{2023-07-08T04:...|              null|  2023-07-08|
|[{31, 44, 2023-07...|{[{2023-07-07T01:...|{1234567890abcdef}|  2023-07-07|
|[{22, 9, 2023-07-...|{[{2023-07-07T21:...|              null|  2023-07-07|
|[{75, 91, 2023-07...|{[{2023-07-06T19:...|{1234567890abcdef}|  2023-07-06|
|[{61, 44, 2023-07...|{[{2023-07-06T13:...|              null|  2023-07-06|
|[{33, 94, 2023-07...|{[{2023-07-09T01:...|              null|  2023-07-09|
|[{24, 17, 2

In [8]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [9]:
from pyspark.sql import functions as f

In [10]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [11]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{24, 37, 2023-07-...|
|{98, 21, 2023-07-...|
|{95, 26, 2023-07-...|
|{66, 71, 2023-07-...|
|{94, 59, 2023-07-...|
|{97, 16, 2023-07-...|
|{88, 20, 2023-07-...|
|{23, 92, 2023-07-...|
|{93, 6, 2023-07-1...|
|{91, 96, 2023-07-...|
|{60, 37, 2023-07-...|
|{37, 60, 2023-07-...|
|{52, 96, 2023-07-...|
|{37, 79, 2023-07-...|
|{94, 76, 2023-07-...|
|{67, 72, 2023-07-...|
|{3, 25, 2023-07-1...|
|{39, 49, 2023-07-...|
|{63, 21, 2023-07-...|
|{92, 29, 2023-07-...|
+--------------------+
only showing top 20 rows



In [12]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [33]:
tweet_df=df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [34]:
tweet_df.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       24|             37|2023-07-10T15:27:...|  0|        95|          9|         58|           71|Tweet fictício ge...|
|       98|             21|2023-07-10T01:22:...| 69|        27|         72|         49|           96|Tweet fictício ge...|
|       95|             26|2023-07-10T06:11:...| 43|        79|         29|         54|           36|Este é um tweet f...|
|       66|             71|2023-07-10T08:02:...| 18|        76|         82|         72|           40|Um terceiro tweet...|
|       94|             59|2023-07-10T22:24:...| 32|        83|         53|         13|           20|Tweet fictício ge...|
|       97|     

In [24]:
df.select(f.explode("includes.users")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [29]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-07-10T04:32:...| 48|User 1|   user1|
|2023-07-10T16:37:...| 61|User 2|   user2|
|2023-07-10T09:00:...| 71|User 3|   user3|
|2023-07-10T03:01:...| 98|User 4|   user4|
|2023-07-10T09:57:...| 59|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [35]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')